In [ ]:
print('Setup complete.')

# Tool-Calling Foundations - Demo (AskSage Version with gpt-5-mini)

**Focus**: tool schemas, read vs write safety, logging tool calls

This notebook demonstrates the fundamentals of tool calling in AI applications, covering proper tool design patterns, safety considerations, and observability practices using AskSage API with gpt-5-mini and HuggingFace embeddings.

## Learning Objectives
- Understand tool schema design and validation
- Distinguish between read and write operations for safety
- Implement proper logging and tracing for tool calls
- Build a foundation for safe tool integration with AskSage using gpt-5-mini

In [ ]:
# Install required packages for Google Colab
!pip install pydantic requests typing-extensions transformers torch huggingface-hub asksageclient

import json
import logging
import time
import traceback
from typing import Dict, List, Any, Optional, Union
from datetime import datetime
from pydantic import BaseModel, Field, validator
import requests

# AskSage client imports
from asksageclient import AskSageClient

# HuggingFace imports for embeddings
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np

# Set up logging for tool call tracing
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Function to load credentials from a JSON file
def load_credentials(filename):
    try:
        with open(filename) as file:
            return json.load(file)
    except FileNotFoundError:
        raise FileNotFoundError("The credentials file was not found.")
    except json.JSONDecodeError:
        raise ValueError("Failed to decode JSON from the credentials file.")

# Load the credentials
credentials = load_credentials('../../credentials.json')

# Extract the API key and email from the credentials
api_key = credentials['credentials']['api_key']
email = credentials['credentials']['Ask_sage_user_info']['username']

# Initialize AskSage client
ask_sage_client = AskSageClient(email, api_key)

# Initialize HuggingFace embedding model (nvidia/NV-Embed-v2)
embedding_model_name = "nvidia/NV-Embed-v2"
try:
    embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name, trust_remote_code=True)
    embedding_model = AutoModel.from_pretrained(embedding_model_name, trust_remote_code=True)
    print("✅ HuggingFace nvidia/NV-Embed-v2 model loaded successfully")
except Exception as e:
    print(f"⚠️  Could not load nvidia/NV-Embed-v2 model: {e}")
    print("Will use mock embeddings for demonstration purposes")
    embedding_tokenizer = None
    embedding_model = None

print("✅ All packages installed, AskSage client initialized, and logging configured!")
print("🚀 This notebook is configured to use gpt-5-mini model")

## 1. Tool Schema Design

Well-defined tool schemas are crucial for reliable AI tool calling. They define the interface, validate inputs, and provide clear documentation.

In [ ]:
# Define tool schemas using Pydantic for validation

class QueryEmbeddingTool(BaseModel):
    """Tool for generating embeddings from text queries using HuggingFace nvidia/NV-Embed-v2 - READ OPERATION"""
    text: str = Field(..., description="Text to generate embedding for", max_length=8000)
    model: str = Field(default="nvidia/NV-Embed-v2", description="HuggingFace embedding model to use")
    
    class Config:
        schema_extra = {
            "example": {
                "text": "What is machine learning?",
                "model": "nvidia/NV-Embed-v2"
            }
        }

class WeatherQueryTool(BaseModel):
    """Tool for querying weather information - READ OPERATION"""
    location: str = Field(..., description="City or location name", min_length=1)
    units: str = Field(default="metric", description="Temperature units: metric, imperial, kelvin")
    
    @validator('units')
    def validate_units(cls, v):
        allowed = ['metric', 'imperial', 'kelvin']
        if v not in allowed:
            raise ValueError(f'Units must be one of {allowed}')
        return v

class AskSageQueryTool(BaseModel):
    """Tool for querying AskSage AI with gpt-5-mini - READ OPERATION"""
    message: str = Field(..., description="Query message for the AI model")
    model: str = Field(default="gpt-5-mini", description="AI model to use")
    system_prompt: str = Field(default="You are a helpful AI assistant", description="System prompt")
    
    class Config:
        schema_extra = {
            "example": {
                "message": "Explain machine learning concepts",
                "model": "gpt-5-mini",
                "system_prompt": "You are a helpful AI assistant"
            }
        }

class FileWriteTool(BaseModel):
    """Tool for writing files - WRITE OPERATION (HIGH RISK)"""
    filename: str = Field(..., description="Name of file to write")
    content: str = Field(..., description="Content to write to file")
    overwrite: bool = Field(default=False, description="Whether to overwrite existing file")
    
    @validator('filename')
    def validate_filename(cls, v):
        import os
        # Basic security check - no path traversal
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid filename: path traversal detected')
        return v

print("Tool schemas defined with validation rules")
print("✅ READ tools: QueryEmbeddingTool (HuggingFace), WeatherQueryTool, AskSageQueryTool (gpt-5-mini)")
print("⚠️  WRITE tools: FileWriteTool (requires safety measures)")

## 2. Read vs Write Safety Classification

Critical distinction: READ operations are generally safe (query data), while WRITE operations can modify state and require additional safety measures.

In [ ]:
class ToolSafetyClassifier:
    """Classify tools by their safety risk level"""
    
    READ_OPERATIONS = {
        'query_embedding': 'Generate text embeddings using HuggingFace',
        'weather_query': 'Query weather data',
        'web_search': 'Search web content',
        'database_read': 'Read from database',
        'api_get': 'GET requests to APIs',
        'asksage_query': 'Query AskSage AI model with gpt-5-mini'
    }
    
    WRITE_OPERATIONS = {
        'file_write': 'Write or modify files',
        'database_write': 'Insert/update database records',
        'api_post': 'POST/PUT requests to APIs',
        'system_command': 'Execute system commands',
        'email_send': 'Send emails or messages'
    }
    
    @classmethod
    def classify_tool(cls, tool_name: str, tool_description: str) -> Dict[str, Any]:
        """Classify a tool's safety level"""
        is_read = any(keyword in tool_description.lower() 
                     for keyword in ['query', 'read', 'get', 'search', 'fetch'])
        is_write = any(keyword in tool_description.lower() 
                      for keyword in ['write', 'create', 'update', 'delete', 'send', 'execute'])
        
        if is_write:
            risk_level = 'HIGH'
            requires_confirmation = True
            operation_type = 'WRITE'
        elif is_read:
            risk_level = 'LOW'
            requires_confirmation = False
            operation_type = 'READ'
        else:
            risk_level = 'MEDIUM'
            requires_confirmation = True
            operation_type = 'UNKNOWN'
        
        return {
            'tool_name': tool_name,
            'operation_type': operation_type,
            'risk_level': risk_level,
            'requires_confirmation': requires_confirmation,
            'safety_notes': f"{'⚠️' if is_write else '✅'} {operation_type} operation"
        }

# Test the classifier
test_tools = [
    ('embed_query', 'Generate embeddings for text queries using nvidia/NV-Embed-v2'),
    ('write_file', 'Write content to a file on disk'),
    ('get_weather', 'Fetch current weather for a location'),
    ('asksage_chat', 'Query AskSage AI model with gpt-5-mini for responses')
]

print("Tool Safety Classification Results:")
print("-" * 50)
for tool_name, description in test_tools:
    result = ToolSafetyClassifier.classify_tool(tool_name, description)
    print(f"Tool: {result['tool_name']}")
    print(f"  Type: {result['operation_type']} | Risk: {result['risk_level']}")
    print(f"  Confirmation needed: {result['requires_confirmation']}")
    print(f"  {result['safety_notes']}\n")

## 3. Tool Call Logging and Tracing

Proper logging is essential for debugging, monitoring, and auditing tool usage in production systems.

In [ ]:
class ToolCallLogger:
    """Enhanced logging for tool calls with tracing and metrics"""
    
    def __init__(self, logger_name: str = "tool_calls"):
        self.logger = logging.getLogger(logger_name)
        self.call_history = []
        self.metrics = {
            'total_calls': 0,
            'successful_calls': 0,
            'failed_calls': 0,
            'read_operations': 0,
            'write_operations': 0
        }
    
    def log_tool_call(self, tool_name: str, parameters: Dict, operation_type: str = "READ"):
        """Log the start of a tool call"""
        call_id = f"{tool_name}_{int(time.time() * 1000)}"
        
        call_record = {
            'call_id': call_id,
            'tool_name': tool_name,
            'operation_type': operation_type,
            'parameters': parameters,
            'timestamp': datetime.now().isoformat(),
            'status': 'STARTED',
            'duration_ms': None,
            'error': None
        }
        
        self.call_history.append(call_record)
        self.metrics['total_calls'] += 1
        self.metrics[f'{operation_type.lower()}_operations'] += 1
        
        self.logger.info(f"🔧 TOOL_CALL_START | {call_id} | {tool_name} | {operation_type}")
        self.logger.debug(f"Parameters: {json.dumps(parameters, default=str)}")
        
        return call_id
    
    def log_tool_success(self, call_id: str, result: Any, duration_ms: float):
        """Log successful tool completion"""
        for record in self.call_history:
            if record['call_id'] == call_id:
                record['status'] = 'SUCCESS'
                record['duration_ms'] = duration_ms
                record['result_size'] = len(str(result)) if result else 0
                break
        
        self.metrics['successful_calls'] += 1
        self.logger.info(f"✅ TOOL_CALL_SUCCESS | {call_id} | {duration_ms:.2f}ms")
    
    def log_tool_error(self, call_id: str, error: Exception, duration_ms: float):
        """Log tool call failure"""
        for record in self.call_history:
            if record['call_id'] == call_id:
                record['status'] = 'ERROR'
                record['duration_ms'] = duration_ms
                record['error'] = str(error)
                break
        
        self.metrics['failed_calls'] += 1
        self.logger.error(f"❌ TOOL_CALL_ERROR | {call_id} | {duration_ms:.2f}ms | {error}")
    
    def get_metrics_summary(self) -> Dict[str, Any]:
        """Get summary of tool call metrics"""
        success_rate = (self.metrics['successful_calls'] / self.metrics['total_calls'] * 100) if self.metrics['total_calls'] > 0 else 0
        
        return {
            **self.metrics,
            'success_rate_percent': round(success_rate, 2),
            'recent_calls': self.call_history[-5:] if self.call_history else []
        }

# Initialize the logger
tool_logger = ToolCallLogger()
print("📊 Tool call logging system initialized")

In [ ]:
# HuggingFace embedding function using nvidia/NV-Embed-v2
def get_huggingface_embedding(text: str, model_name: str = "nvidia/NV-Embed-v2") -> List[float]:
    """Generate embeddings using HuggingFace nvidia/NV-Embed-v2 model"""
    global embedding_model, embedding_tokenizer
    
    if embedding_model is None or embedding_tokenizer is None:
        # Fallback to mock embedding if model not available
        import random
        embedding_dim = 4096  # nvidia/NV-Embed-v2 dimension
        return [random.uniform(-1, 1) for _ in range(embedding_dim)]
    
    try:
        # Tokenize the input text
        inputs = embedding_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        
        # Generate embeddings
        with torch.no_grad():
            outputs = embedding_model(**inputs)
            
        # Extract embeddings (usually from last_hidden_state)
        # Mean pooling across sequence length
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
        
        # Convert to list
        return embeddings.tolist()
        
    except Exception as e:
        print(f"Error generating embedding: {e}")
        # Fallback to mock embedding
        import random
        embedding_dim = 4096  # nvidia/NV-Embed-v2 dimension
        return [random.uniform(-1, 1) for _ in range(embedding_dim)]

# Updated embed_query function with HuggingFace and logging
def embed_query(text: str, model: str = "nvidia/NV-Embed-v2") -> List[float]:
    """Embedding function using HuggingFace nvidia/NV-Embed-v2 with logging"""
    
    # Validate input using our schema
    tool_params = QueryEmbeddingTool(text=text, model=model)
    
    # Log tool call start
    call_id = tool_logger.log_tool_call(
        tool_name="embed_query",
        parameters=tool_params.dict(),
        operation_type="READ"
    )
    
    start_time = time.time()
    
    try:
        # Generate embedding using HuggingFace model
        embedding = get_huggingface_embedding(text, model)
        
        # Log success
        duration_ms = (time.time() - start_time) * 1000
        tool_logger.log_tool_success(call_id, embedding, duration_ms)
        
        return embedding
        
    except Exception as e:
        duration_ms = (time.time() - start_time) * 1000
        tool_logger.log_tool_error(call_id, e, duration_ms)
        raise

# Test the function
print("Testing embed_query with HuggingFace nvidia/NV-Embed-v2 and logging:")
test_embedding = embed_query("What is artificial intelligence?")
print(f"Generated embedding with {len(test_embedding)} dimensions")
print(f"First 5 values: {test_embedding[:5]}")

# Show metrics
print("\nCurrent metrics:")
metrics = tool_logger.get_metrics_summary()
for key, value in metrics.items():
    if key != 'recent_calls':
        print(f"  {key}: {value}")

In [ ]:
# Example: AskSage query function with gpt-5-mini and logging
def query_asksage(prompt: str, model: str = "gpt-5-mini", system_prompt: Optional[str] = None) -> str:
    """Query AskSage AI model with gpt-5-mini and logging"""
    
    # Validate input using our schema
    tool_params = AskSageQueryTool(message=prompt, model=model, system_prompt=system_prompt)
    
    # Log tool call start
    call_id = tool_logger.log_tool_call(
        tool_name="query_asksage",
        parameters=tool_params.dict(),
        operation_type="READ"
    )
    
    start_time = time.time()
    
    try:
        # Use AskSage client to get response with gpt-5-mini
        query_params = {
            'message': prompt,
            'model': model
        }
        
        if system_prompt:
            query_params['system_prompt'] = system_prompt
            
        response = ask_sage_client.query(**query_params)
        
        # Extract the actual response text based on AskSage API response format
        if isinstance(response, dict) and 'response' in response:
            response_text = response['response']
        elif isinstance(response, dict) and 'ret' in response:
            response_text = response['ret']
        else:
            response_text = str(response)
        
        # Log success
        duration_ms = (time.time() - start_time) * 1000
        tool_logger.log_tool_success(call_id, response_text, duration_ms)
        
        return response_text
        
    except Exception as e:
        duration_ms = (time.time() - start_time) * 1000
        tool_logger.log_tool_error(call_id, e, duration_ms)
        
        # Return a fallback response for demo purposes
        fallback_response = f"AskSage query with gpt-5-mini failed: {str(e)}. This is a demo fallback response for prompt: '{prompt}'"
        return fallback_response

# Test AskSage query with gpt-5-mini
print("Testing AskSage query with gpt-5-mini and logging:")
test_response = query_asksage(
    prompt="Explain the concept of machine learning in simple terms.",
    model="gpt-5-mini",
    system_prompt="You are a helpful AI assistant that explains complex topics in simple terms."
)
print(f"AskSage gpt-5-mini response: {test_response[:200]}{'...' if len(test_response) > 200 else ''}")

# Show updated metrics
print("\nUpdated metrics after AskSage gpt-5-mini query:")
metrics = tool_logger.get_metrics_summary()
for key, value in metrics.items():
    if key != 'recent_calls':
        print(f"  {key}: {value}")

# Test with a more complex query
print("\n" + "="*50)
print("Testing gpt-5-mini with a more complex reasoning task:")
complex_response = query_asksage(
    prompt="Compare the advantages and disadvantages of supervised vs unsupervised learning. Provide specific examples.",
    model="gpt-5-mini",
    system_prompt="You are an expert in machine learning. Provide detailed, structured responses with clear examples."
)
print(f"Complex query response: {complex_response[:300]}{'...' if len(complex_response) > 300 else ''}")

## Summary

In this demo, we covered the foundations of tool calling with AskSage gpt-5-mini and HuggingFace:

1. **Tool Schema Design**: Using Pydantic for validation and clear interfaces
2. **Safety Classification**: Distinguishing READ (safe) vs WRITE (risky) operations
3. **Logging & Tracing**: Comprehensive monitoring for debugging and auditing
4. **AskSage gpt-5-mini Integration**: Using AskSageClient with gpt-5-mini for AI model queries
5. **HuggingFace Embeddings**: Using nvidia/NV-Embed-v2 for text embeddings

These patterns provide a solid foundation for building reliable, observable, and safe AI tool systems.

### Key Takeaways
- Always validate tool inputs with proper schemas
- Classify operations by risk level and apply appropriate safety measures
- Implement comprehensive logging for observability and debugging
- READ operations are generally safe; WRITE operations require extra caution
- AskSage with gpt-5-mini provides an alternative to OpenAI for advanced AI model queries
- gpt-5-mini offers improved reasoning capabilities for complex tasks
- HuggingFace models like nvidia/NV-Embed-v2 offer powerful embedding capabilities

### Setup Requirements
- Ensure you have `../../credentials.json` with proper AskSage API credentials
- Install required packages: `asksageclient`, `transformers`, `torch`, `pydantic`
- The notebook will fallback to mock embeddings if nvidia/NV-Embed-v2 cannot be loaded
- gpt-5-mini model must be available in your AskSage account

### gpt-5-mini Specific Notes
- gpt-5-mini is configured as the default model for all AskSage queries
- The model provides enhanced reasoning capabilities compared to earlier versions
- All tool functions have been updated to explicitly use gpt-5-mini
- System prompts are properly integrated for better model guidance